In [9]:

import os
import geopandas as gpd
import osmnx as ox

In [10]:
os.makedirs("../data/processed", exist_ok=True)

In [11]:
# --- 1) load Mitte boundary 
mitte_fp = "../maps/mitte.geojson"
mitte = gpd.read_file(mitte_fp)
print("Loaded Mitte boundary:", mitte.shape)


Loaded Mitte boundary: (1, 7)


In [12]:
# ensure geometric validity
mitte = mitte.to_crs(epsg=25833)  # metric CRS for Berlin
print("Mitte CRS:", mitte.crs)

Mitte CRS: EPSG:25833


In [13]:
# --- 2) loading building shapefile (ALKIS) from maps/ ---
buildings_shapefile = "../maps/gebaeude.shp"   #
if not os.path.exists(buildings_shapefile):
    raise FileNotFoundError(f"Cannot find buildings shapefile at {buildings_shapefile}. Put gebaeude.* files into maps/")
buildings = gpd.read_file(buildings_shapefile)
print("Raw buildings loaded:", buildings.shape)
print("Building columns:", list(buildings.columns)[:12])

Raw buildings loaded: (760088, 24)
Building columns: ['gfk', 'bezgfk', 'ofl', 'bezofl', 'aog', 'aug', 'hoh', 'bat', 'bezbat', 'nam', 'baw', 'bezbaw']


In [14]:
# reproject to metric CRS (EPSG:25833) for accurate spatial ops
buildings = buildings.to_crs(epsg=25833)
print("Buildings reprojected. CRS now:", buildings.crs)

Buildings reprojected. CRS now: EPSG:25833


In [15]:
# --- 3) cliping buildings to Mitte ---
# Using gpd.clip which is robust; this keeps only building geometry within Mitte polygon(s)
buildings_mitte = gpd.clip(buildings, mitte)
print("Buildings inside Mitte:", buildings_mitte.shape)

Buildings inside Mitte: (48978, 24)


In [16]:
# Saving processed buildings (GeoJSON) for later steps
out_bld_fp = "../data/processed/buildings_mitte.geojson"
buildings_mitte.to_file(out_bld_fp, driver="GeoJSON")
print("Saved clipped buildings to:", out_bld_fp)

Saved clipped buildings to: ../data/processed/buildings_mitte.geojson


In [17]:
# --- 4) geting public spaces from OSM (parks, benches, squares, playgrounds) ---
tags = {
    "leisure": ["park", "garden", "playground"],
    "amenity": ["bench", "shelter"],
    "public_transport": ["platform"],
    "landuse": ["recreation_ground", "grass"]
}

In [19]:
# Using the Mitte polygon for spatial query
mitte_poly = mitte.geometry.iloc[0]

# NEW OSMnx syntax (v2.x)
osm_gdf = ox.features.features_from_polygon(mitte_poly, tags)

print("OSM raw geometries fetched:", osm_gdf.shape)

AttributeError: 'Point' object has no attribute 'geoms'

In [1]:
# Notebook: 02_load_buildings_and_public_spaces.ipynb

import os
import geopandas as gpd
import osmnx as ox
from shapely.geometry import MultiPolygon, Polygon

# --- create processed folder if missing ---
os.makedirs("../data/processed", exist_ok=True)

# --- 1) load Mitte boundary (we created this earlier) ---
mitte_fp = "../maps/mitte.geojson"
mitte = gpd.read_file(mitte_fp)
print("Loaded Mitte boundary:", mitte.shape)

# ensure geometric validity
mitte = mitte.to_crs(epsg=25833)  # metric CRS for Berlin
print("Mitte CRS:", mitte.crs)

# --- 2) load building shapefile (ALKIS) from maps/ ---
buildings_shapefile = "../maps/gebaeude.shp"
if not os.path.exists(buildings_shapefile):
    raise FileNotFoundError(f"Cannot find buildings shapefile at {buildings_shapefile}. Put gebaeude.* files into maps/")

buildings = gpd.read_file(buildings_shapefile)
print("Raw buildings loaded:", buildings.shape)
print("Building columns:", list(buildings.columns)[:12])  # show first columns

# reproject to metric CRS (EPSG:25833) for accurate spatial ops
buildings = buildings.to_crs(epsg=25833)
print("Buildings reprojected. CRS now:", buildings.crs)

# --- 3) clip buildings to Mitte ---
buildings_mitte = gpd.clip(buildings, mitte)
print("Buildings inside Mitte:", buildings_mitte.shape)

# Save processed buildings (GeoJSON) for later steps
out_bld_fp = "../data/processed/buildings_mitte.geojson"
buildings_mitte.to_file(out_bld_fp, driver="GeoJSON")
print("Saved clipped buildings to:", out_bld_fp)

# --- 4) get public spaces from OSM (parks, benches, squares, playgrounds) ---
tags = {
    "leisure": ["park", "garden", "playground"],
    "amenity": ["bench", "shelter"],
    "public_transport": ["platform"],
    "landuse": ["recreation_ground", "grass"]
}

# --- Important fix: use EPSG:4326 (lat/lon) for OSMnx ---
mitte_poly_latlon = mitte.to_crs(epsg=4326).geometry.iloc[0]

# Ensure we have a MultiPolygon
if isinstance(mitte_poly_latlon, Polygon):
    mitte_poly_latlon = MultiPolygon([mitte_poly_latlon])

# Fetch OSM geometries
osm_gdf = ox.features.features_from_polygon(mitte_poly_latlon, tags)
print("OSM raw geometries fetched:", osm_gdf.shape)

# Keep only relevant columns
keep_cols = ["geometry", "name", "leisure", "amenity", "landuse"]
public_spaces = osm_gdf.reset_index()[[c for c in keep_cols if c in osm_gdf.columns or c=="geometry"]].copy()

# Reproject back to metric CRS (EPSG:25833) for consistency with buildings
public_spaces = public_spaces.set_geometry("geometry").to_crs(epsg=25833)
print("Public spaces (filtered columns):", public_spaces.shape)

# Save public spaces
out_public_fp = "../data/processed/mitte_public_spaces.geojson"
public_spaces.to_file(out_public_fp, driver="GeoJSON")
print("Saved public spaces to:", out_public_fp)

print("STEP COMPLETE: buildings_mitte and mitte_public_spaces saved in data/processed/")


DataSourceError: ../maps/mitte.geojson: No such file or directory